# Model_Training

##### 1.1 Import Machining Libraries

In [17]:
# Basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# Modeling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Importing models
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostRegressor
from xgboost import XGBClassifier
import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#### import data into csv

In [2]:
df = pd.read_csv('Data/studentsPerformance.csv')

In [3]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## Preparing X and Y features

In [4]:
x = df.drop(columns=['math score'], axis=1)

In [5]:
x.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
print(x.shape)

(1000, 7)


In [7]:
print(x.columns)

Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course', 'reading score', 'writing score'],
      dtype='object')


In [9]:
print("Categories in 'gender' variable:", end=" ")
print(df['gender'].unique())

print("Categories in 'race/ethnicity' variable:", end=" ")
print(df['race/ethnicity'].unique())

print("Categories in 'parental level of education' variable:", end=" ")
print(df['parental level of education'].unique())

print("Categories in 'lunch' variable:", end=" ")
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:", end=" ")
print(df['test preparation course'].unique())

Categories in 'gender' variable: ['female' 'male']
Categories in 'race/ethnicity' variable: ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in 'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable: ['standard' 'free/reduced']
Categories in 'test preparation course' variable: ['none' 'completed']


In [10]:
y = df['math score']

In [11]:
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

In [12]:
# Creating column transformer with 3 transformers
num_features = x.select_dtypes(exclude=['object']).columns
cat_features = x.select_dtypes(include=['object']).columns

numeric_transformer = StandardScaler()
ohe_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', ohe_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_features)
    ]
)

In [13]:
X = preprocessor.fit_transform(x)

X.shape

(1000, 19)

In [14]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [15]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(800, 19) (800,)
(200, 19) (200,)


#### Create Evaluate Function to give all metrics after training models

In [16]:
def evaluate_model(true,predicted):
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    
    return mae, mse,rmse, r2

In [25]:
# create models to see which one is the best
models = [
    'K-NeighborsRegressor', KNeighborsRegressor(),
    'DecisionTreeRegressor', DecisionTreeRegressor(),
    'RandomForestRegressor', RandomForestRegressor(),
    'AdaBoostRegressor', AdaBoostRegressor(),
    'LinearRegression', LinearRegression(),
    'Ridge', Ridge(),
    'Lasso', Lasso(),
    'XGBRegressor', XGBRegressor(),
    'CatBoostRegressor', CatBoostRegressor(verbose=0),
]

model_list = []
r2_list = []

# Loop through the models and evaluate them
for i in range(len(list(models))):
    model = list(model.values())[i]
    #Train models
    model.fit(X_train, y_train)

    #Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate the model
    model_train_mae,model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae,model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    # Print the evaluation metrics
    print(list(model.keys())[i])

    model_list.append(list(model.keys())[i])

    print("Mode Performance for Train Data")
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    r2_list.append(model_train_r2)
    
    print("---------------------------------------")
    
    print("Mode Performance for Test Data")
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print("="*35)
    print("\n")

AttributeError: 'KNeighborsRegressor' object has no attribute 'values'

In [ ]:
def fit_predict_evaluate(models, X_train, X_test, y_train, y_test):
    results = {}
    
    for model_name, model in models:
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predict on training and testing data
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # Evaluate the model
        train_mae = mean_absolute_error(y_train, y_train_pred)
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_rmse = np.sqrt(train_mse)
        train_r2 = r2_score(y_train, y_train_pred)
        
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score(y_test, y_test_pred)
        
        # Store the results
        results[model_name] = {
            'Train MAE': float(train_mae),
            'Train MSE': float(train_mse),
            'Train RMSE': float(train_rmse),
            'Train R2': float(train_r2),
            'Test MAE': float(test_mae),
            'Test MSE': float(test_mse),
            'Test RMSE': float(test_rmse),
            'Test R2': float(test_r2)
        }
    
    # Find the best model based on Test R2 score
    best_model_name = max(results, key=lambda model: results[model]['Test R2'])
    best_model_metrics = results[best_model_name]
    
    # Print the performance of the best model
    print(f"Best Model: {best_model_name}")
    print("Training Performance:")
    print(f"MAE: {best_model_metrics['Train MAE']:.2f}, MSE: {best_model_metrics['Train MSE']:.2f}, RMSE: {best_model_metrics['Train RMSE']:.2f}, R2: {best_model_metrics['Train R2']:.2f}")
    print("Testing Performance:")
    print(f"MAE: {best_model_metrics['Test MAE']:.2f}, MSE: {best_model_metrics['Test MSE']:.2f}, RMSE: {best_model_metrics['Test RMSE']:.2f}, R2: {best_model_metrics['Test R2']:.2f}")
    
    return results
